# Compare Runner

Loads one vanilla CSV and one RAG CSV (generated by the other notebooks), aligns by `qid`, and reports overall accuracy + uplift. Also shows where RAG helped/hurt.

**You only need to edit the PATHS cell.**

In [ ]:
!pip -q install -U pandas

In [ ]:
# ===== PATHS (EDIT ME) =====
VANILLA_CSV = "/content/drive/MyDrive/your_project/outputs/vanilla/vanilla_YYYYMMDD_HHMMSS.csv"  # <- TODO
RAG_CSV     = "/content/drive/MyDrive/your_project/outputs/rag/rag_YYYYMMDD_HHMMSS.csv"          # <- TODO
OUT_CSV     = "/content/drive/MyDrive/your_project/outputs/comparison.csv"                       # <- TODO

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
van = pd.read_csv(VANILLA_CSV)
rag = pd.read_csv(RAG_CSV)

# Basic checks
for df, name in [(van,"vanilla"), (rag,"rag")]:
    assert "qid" in df.columns, f"{name} missing qid"
    assert "correct" in df.columns, f"{name} missing correct"

# Keep minimal columns for merge
van_m = van[["qid","parsed_choice","gold_choice","correct"]].rename(columns={
    "parsed_choice":"vanilla_choice",
    "correct":"vanilla_correct",
})
rag_m = rag[["qid","parsed_choice","gold_choice","correct","retrieved_ids","context_chars"]].rename(columns={
    "parsed_choice":"rag_choice",
    "correct":"rag_correct",
})

merged = van_m.merge(rag_m, on=["qid","gold_choice"], how="inner")

merged["delta"] = merged["rag_correct"].fillna(0) - merged["vanilla_correct"].fillna(0)

van_acc = merged["vanilla_correct"].mean()
rag_acc = merged["rag_correct"].mean()
uplift = rag_acc - van_acc

van_acc, rag_acc, uplift, len(merged)

In [ ]:
print(f"Vanilla accuracy: {van_acc:.3f}")
print(f"RAG accuracy:     {rag_acc:.3f}")
print(f"Uplift:           {uplift:+.3f}")

In [ ]:
# Where RAG helped / hurt
helped = merged[merged["delta"] == 1].copy()
hurt   = merged[merged["delta"] == -1].copy()

print("Helped:", len(helped))
print("Hurt:  ", len(hurt))

helped.head(10)

In [ ]:
# Save merged comparison
merged.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("Saved:", OUT_CSV)

In [ ]:
# Optional: quick bar chart (no fixed colors)
import matplotlib.pyplot as plt

labels = ["Vanilla", "RAG"]
values = [van_acc, rag_acc]

plt.figure(figsize=(5,4))
plt.bar(labels, values)
plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.title("Accuracy Comparison")
plt.show()